In [ ]:
"""
依赖自检
"""

import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

from diffusion.env import ensure_dependencies

ensure_dependencies()

import torch
import torchvision
from torch import nn
from torch.nn import functional as f
from torch.utils.data import DataLoader
from diffusers import DDPMScheduler, UNet2DModel
from matplotlib import pyplot as plt


In [ ]:
"""
Device 自检
"""

from diffusion.env import select_device

device = select_device(torch)


In [ ]:
"""
数据集测试
"""

from diffusion.data import create_dataloader, create_mnist_dataset

# 数据集
dataset = create_mnist_dataset(
    root="../data/datasets",
    train=True,
    download=True,
)

batch_size = 16

# 为数据集创建数据加载器
dataset_loader = create_dataloader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
)

# 从加载器中取出第一批数据
x, y = next(iter(dataset_loader))
print('Input shape:', x.shape)
print('Labels :', y)
plt.imshow(torchvision.utils.make_grid(x)[0], cmap = 'gray')  # 以单通道取出所有图像，拼接成大图并用灰度显示


In [ ]:
"""
添加噪声，并对输出结果进行可视化
"""

from diffusion.noise import corrupt

# 绘制输入数据
fig, axs = plt.subplots(2, 1, figsize=(12, 5))  # 画布行数，画布列数，画布大小。plt.subplots返回两个方法，第一个是画布对象fig，第二个是子图对象axs
plt.subplots_adjust(hspace=0.4)  # 扩大子图间距
axs[0].set_title('Input Images')
axs[0].imshow(torchvision.utils.make_grid(x)[0], cmap='gray')  # 以单通道取出所有图像，拼接成大图并用灰度显示

# 加入噪声
amount = torch.linspace(0, 1, x.shape[0])  # 在指定的范围内，生成一组等距离的数字，数量与x的Batch_size相同
noised_x = corrupt(x, amount)

# 绘制加入噪声后的图像
axs[1].set_title('Corrupted Images (--- amount increases --->)')
axs[1].imshow(torchvision.utils.make_grid(noised_x)[0], cmap='gray')


In [ ]:
"""
部署 UNet 网络
"""

from diffusion.models import BasicUnet


In [ ]:
"""
验证输入和输出的形状是否相同，并查看 UNet 网络的参数量
"""

net = BasicUnet()
x = torch.rand(8, 1, 28, 28)  # 生成形状 (8,1,28,28) 的随机张量
y = net(x).shape  # 将随机张量丢进网络，查看输出形状是否与输入相同
print(y)

sum(p.numel() for p in net.parameters())  # 计算网络的参数量

In [ ]:
"""
开始训练模型
"""

batch_size = 64

# 数据加载器
dataset_loader = DataLoader(
                dataset,                  # 要加载的数据对象
                batch_size = batch_size,  # 每次迭代加载的样本数量
                shuffle=True              # 打乱数据顺序
                )

# 运行周期
num_epochs = 5

# 创建 UNet 网络
net = BasicUnet().to(device)

# 损失函数（均方误差）
loss_fn = nn.MSELoss()

# 优化器，根据损失函数结果调整网络权重
optimizer = torch.optim.AdamW(net.parameters(), lr=1e-3)  # 学习率：1e-3


# 记录训练损失
loss_history = []

# 训练循环
for epoch in range(num_epochs):
    for x, y in dataset_loader:
        # 加载数据并添加噪声
        x = x.to(device)  # 加载数据
        noise_amount = torch.rand(x.shape[0]).to(device)  # 为每个样本生成一个随机的噪声数量
        noisy_x = corrupt(x, noise_amount)  # 向样本中添加噪声

        # 预测的噪声结果
        predicted_image = net(noisy_x)

        # 计算损失
        loss = loss_fn(predicted_image, x)  # 对比预测噪声与原始图像

        # 反向传播并更新权重
        optimizer.zero_grad()  # 清除之前的梯度
        loss.backward()        # 反向传播计算新的梯度
        optimizer.step()       # 更新权重

        # 记录损失
        loss_history.append(loss.item())

    # 输出每个 epoch 的损失均值
    avg_loss = sum(loss_history[-len(dataset_loader):]) / len(dataset_loader)
    print(f"Finished epoch {epoch}. Average loss: {avg_loss:05f}")


# 绘制损失曲线
plt.plot(loss_history)
plt.ylim(0, 0.1)

In [ ]:
"""
观察训练结果
"""

x, y = next(iter(dataset_loader))  # 从数据集中取出一批数据
x = x[:8]  # 取出前8个样本

amount = torch.linspace(0, 1, x.shape[0])  # 生成一组等距离的噪声数量
noised_x = corrupt(x, amount)  # 向样本中添加噪

# 得到模型预测结果
with torch.no_grad():  # 在评估模式下，不计算梯度
    predicted_image = net(noised_x.to(device)).cpu()  # 预测噪声，将结果移回CPU（NumPy无法绘制GPU数据）

# 绘制结果
fig, axs = plt.subplots(3, 1, figsize=(12, 7))
axs[0].set_title('Input Images')
axs[0].imshow(torchvision.utils.make_grid(x)[0].clip(0, 1), cmap='gray')
axs[1].set_title('Corrupted Images (--- amount increases --->)')
axs[1].imshow(torchvision.utils.make_grid(noised_x)[0].clip(0, 1), cmap='gray')
axs[2].set_title('Predicted Noise')
axs[2].imshow(torchvision.utils.make_grid(predicted_image)[0].clip(0, 1), cmap='gray')

In [ ]:
"""
拆解采样步骤
"""

step = 5
x = torch.rand(8, 1, 28, 28).to(device)  # 随机初始化一个图像张量
step_history = [x.detach().cpu()]  # 每个步骤的图像
predicted_output = []  # 每个步骤的预测输出

for i in range(step):
    with torch.no_grad():
        predicted_image = net(x)  # 预测噪声
        predicted_output.append(predicted_image.detach().cpu())  # 记录预测输出

        mix_factor = 1/(step - i)  # 朝预测方向移动的步骤
        x = x * (1 - mix_factor) + predicted_image * mix_factor  # 更新图像
        step_history.append(x.detach().cpu())  # 记录当前步骤的图像

# 绘制每个步骤的图像和预测输出
fig, axs = plt.subplots(step, 2, figsize=(9, 4), sharex=True)
axs[0, 0].set_title('Input Image')
axs[0, 1].set_title('Predicted Noise')
for i in range(step):
    axs[i, 0].imshow(torchvision.utils.make_grid(step_history[i])[0].clip(0, 1), cmap='gray')
    axs[i, 1].imshow(torchvision.utils.make_grid(predicted_output[i])[0].clip(0, 1), cmap='gray')